## **Deep Learning avec LSTM (mémoire à long terme) :**

In [10]:
import os  
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

from pathlib import Path
sns.set_theme(style="whitegrid", palette="pastel")
from matplotlib import rcParams

import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NAZIFOU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / 'Data' 

df = pd.read_csv(DATA_DIR / 'Top_1000_IMDb_movies.csv')
df.head(3)

,Unnamed: 0,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Gross,Votes,Description
0,0,The Shawshank Redemption,1994,142,9.3,82.0,28.34,"27,77,378","Over the course of several years, two convicts..."
1,1,The Godfather,1972,175,9.2,100.0,134.97,"19,33,588","Don Vito Corleone, head of a mafia family, dec..."
2,2,The Dark Knight,2008,152,9.0,84.0,534.86,"27,54,087",When the menace known as the Joker wreaks havo...


In [12]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns

Index(['unnamed:_0', 'movie_name', 'year_of_release', 'watch_time',
       'movie_rating', 'metascore_of_movie', 'gross', 'votes', 'description'],
      dtype='object')

## **VADER**

In [ ]:
#Remove Stopwords
stop_words = set(stopwords.words('english'))
 
# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

#Clean Text
def clean_text(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    return text

#stemming
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
df['description'] = df['description'].astype(str)
df['description'] = df['description'].apply(lambda x: remove_stopwords(x))
df['description'] = df['description'].apply(lambda x:clean_text(x))
df['description'] = df['description'].apply(stemming)

KeyError: 'Description'

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Calculate sentiment scores
df['VADER_Sentiment'] = df['Description'].apply(lambda text: sia.polarity_scores(text)['compound'])

# Categorize sentiment
df['VADER_Sentiment_Label'] = df['VADER_Sentiment'].apply(lambda score: 'Positive' if score > 0 else ('Negative' if score < 0 else 'Neutral'))

# Print sample results
print(df[['Description', 'VADER_Sentiment_Label']].sample(10))

In [ ]:
from sklearn.model_selection import train_test_split
# Diviser l'ensemble de données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(df['description'], df['VADER_Sentiment_Label'], test_size=0.2, random_state=42)

In [ ]:

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_sequences, maxlen=200, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=200, padding='post', truncating='post')

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert sentiment labels to categorical
y_train_categorical = pd.get_dummies(y_train)
y_test_categorical = pd.get_dummies(y_test)

# Train the model
model.fit(X_train_padded, y_train_categorical, validation_data=(X_test_padded, y_test_categorical), epochs=10, batch_size=32)

NameError: name 'X_train' is not defined